# 🚀 MEJORA DEL MODELO PARA TIEMPO REAL

## Problemas Identificados del Modelo Actual:
1. **Dataset limitado**: Solo 200 muestras, 6 clases
2. **Arquitectura simple**: CNN básica, no optimizada para detección
3. **Resolución baja**: 128x128 píxeles
4. **Una sola detección**: No maneja múltiples objetos por imagen
5. **Sin optimización de velocidad**: No preparado para tiempo real

## Plan de Mejora (Fases):
1. ✅ **Expandir Dataset** (Fase actual)
2. 🔄 **Mejorar Arquitectura** (Transfer Learning)
3. 🔄 **Optimizar Preprocesamiento**
4. 🔄 **Integrar Cámara**
5. 🔄 **Optimizar Inferencia en Tiempo Real**
 

In [18]:
# 🎯 FASE 1: EXPANSIÓN DEL DATASET
import fiftyone.zoo as foz
import fiftyone as fo

# Verificar todas las clases disponibles en Open Images
print("Verificando clases disponibles...")
available_classes = fo.utils.openimages.get_classes()
print(f"Total de clases disponibles: {len(available_classes)}")

# Seleccionar clases más comunes para cámara en tiempo real
COMMON_OBJECTS = [
    "Person", "Car", "Chair", "Table", "Book", "Laptop", 
    "Cell phone", "Bottle", "Cup", "Bowl", "Apple", "Orange",
    "Cat", "Dog", "Bird", "Backpack", "Handbag", "Clock",
    "Keyboard", "Mouse", "Television", "Microwave", "Refrigerator"
]

# Filtrar clases que realmente existen en el dataset
valid_classes = available_classes
print(f"Clases válidas seleccionadas: {len(valid_classes)}")
print("Clases:", valid_classes[:10], "..." if len(valid_classes) > 10 else "")


Verificando clases disponibles...
Total de clases disponibles: 601
Clases válidas seleccionadas: 601
Clases: ['Accordion', 'Adhesive tape', 'Aircraft', 'Airplane', 'Alarm clock', 'Alpaca', 'Ambulance', 'Animal', 'Ant', 'Antelope'] ...


In [19]:
# 📈 DESCARGAR DATASET EXPANDIDO
print("Descargando dataset expandido...")

# Descargar conjunto más grande con más clases
expanded_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",  # Usar split de entrenamiento (más datos)
    label_types=["detections"],
    classes=valid_classes[:30],  # Usar las primeras 15 clases válidas
    max_samples=2500,  # Aumentar a 2000 muestras
    dataset_name="expanded_object_detection_dataset"
)

print(f"Dataset expandido - Muestras: {len(expanded_dataset)}")
print(f"Clases incluidas: {valid_classes[:15]}")

# Análisis del dataset
print("\n📊 ANÁLISIS DEL DATASET:")
print("=" * 50)

# Contar detecciones por clase
class_counts = {}
total_detections = 0

for sample in expanded_dataset:
    if hasattr(sample, 'ground_truth') and sample.ground_truth:
        for detection in sample.ground_truth.detections:
            label = detection.label
            class_counts[label] = class_counts.get(label, 0) + 1
            total_detections += 1

print(f"Total de detecciones: {total_detections}")
print("\nDistribución por clase:")
for label, count in sorted(class_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {label}: {count} detecciones")


Descargando dataset expandido...
Found 80 images, downloading the remaining 2420
 100% |█████████████████| 2420/2420 [1.2m elapsed, 0s remaining, 16.5 files/s]      
Dataset info written to 'C:\Users\Personal\fiftyone\open-images-v6\info.json'
Loading existing dataset 'expanded_object_detection_dataset'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Dataset expandido - Muestras: 2000
Clases incluidas: ['Accordion', 'Adhesive tape', 'Aircraft', 'Airplane', 'Alarm clock', 'Alpaca', 'Ambulance', 'Animal', 'Ant', 'Antelope', 'Apple', 'Armadillo', 'Artichoke', 'Auto part', 'Axe']

📊 ANÁLISIS DEL DATASET:
Total de detecciones: 18373

Distribución por clase:
  Person: 3437 detecciones
  Car: 1335 detecciones
  Chair: 905 detecciones
  Wheel: 592 detecciones
  Footwear: 527 detecciones
  Bird: 465 detecciones
  Bottle: 458 detecciones
  Table: 413 detecciones
  Tree: 397 detecciones
  Mammal: 388 detecciones
  Window: 386 detecciones
  Clothi

In [20]:
# 🔄 PREPROCESAMIENTO MEJORADO PARA TIEMPO REAL
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from collections import Counter

# SOLUCIÓN AL ERROR: Filtrar clases con pocas muestras
MIN_SAMPLES_PER_CLASS = 10  # Mínimo 10 muestras por clase

print("🔍 Analizando distribución de clases...")
print(f"Classes with minimum samples ({MIN_SAMPLES_PER_CLASS}):")

# Filtrar clases que tienen suficientes muestras
valid_classes_filtered = [cls for cls, count in class_counts.items() 
                         if count >= MIN_SAMPLES_PER_CLASS]

print(f"   - Clases originales: {len(class_counts)}")
print(f"   - Clases filtradas (>={MIN_SAMPLES_PER_CLASS} muestras): {len(valid_classes_filtered)}")
print(f"   - Top clases: {valid_classes_filtered[:20]}")

# Configuración mejorada
IMG_SIZE = 224  # Aumentar resolución para mejor precisión
NEW_CLASSES = valid_classes_filtered[:25]  # Usar top 25 clases con suficientes datos
class_to_idx = {c: i for i, c in enumerate(NEW_CLASSES)}
idx_to_class = {i: c for c, i in class_to_idx.items()}

print(f"\n🎯 Configuración actualizada:")
print(f"   - Resolución: {IMG_SIZE}x{IMG_SIZE}")
print(f"   - Número de clases: {len(NEW_CLASSES)}")
print(f"   - Clases seleccionadas: {NEW_CLASSES}")

# Preprocesamiento optimizado
def preprocess_image(img_path, target_size=(IMG_SIZE, IMG_SIZE)):
    """Preprocesamiento optimizado para tiempo real"""
    img = cv2.imread(img_path)
    if img is None:
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    
    return img

# Extraer datos con filtrado de clases
images, labels, bboxes = [], [], []
sample_info = []

print("\n🔄 Procesando dataset con clases filtradas...")
processed_count = 0
skipped_count = 0

for sample in expanded_dataset:
    if not hasattr(sample, 'ground_truth') or not sample.ground_truth:
        continue
        
    img = preprocess_image(sample.filepath)
    if img is None:
        continue
    
    # Procesar cada detección en la imagen
    valid_detections = 0
    for detection in sample.ground_truth.detections:
        if detection.label in class_to_idx:  # Solo clases filtradas
            # Extraer bounding box
            bbox = detection.bounding_box  # [x, y, width, height] en formato relativo
            x, y, w, h = bbox
            
            # Convertir a formato [ymin, xmin, ymax, xmax]
            ymin, xmin = y, x
            ymax, xmax = y + h, x + w
            
            # Validar bounding box
            if 0 <= xmin < xmax <= 1 and 0 <= ymin < ymax <= 1:
                images.append(img)
                labels.append(class_to_idx[detection.label])
                bboxes.append([ymin, xmin, ymax, xmax])
                sample_info.append({
                    'filepath': sample.filepath,
                    'label': detection.label,
                    'bbox': [ymin, xmin, ymax, xmax]
                })
                valid_detections += 1
        else:
            skipped_count += 1
    
    if valid_detections > 0:
        processed_count += 1
        if processed_count % 100 == 0:
            print(f"   Procesadas {processed_count} imágenes...")

# Convertir a arrays numpy
images = np.array(images)
labels = np.array(labels)
bboxes = np.array(bboxes)

print(f"\n✅ DATASET PROCESADO:")
print(f"   - Forma de imágenes: {images.shape}")
print(f"   - Forma de etiquetas: {labels.shape}")
print(f"   - Forma de bboxes: {bboxes.shape}")
print(f"   - Muestras totales: {len(images)}")
print(f"   - Detecciones omitidas: {skipped_count}")

# Verificar distribución final
label_counts = Counter(labels)
print(f"\n📊 DISTRIBUCIÓN FINAL POR CLASE:")
for i, (label_idx, count) in enumerate(label_counts.most_common()):
    class_name = idx_to_class[label_idx]
    print(f"   {i+1}. {class_name}: {count} muestras")
    if i >= 9:  # Mostrar solo top 10
        break

# División en entrenamiento, validación y prueba (SIN stratify si aún hay problemas)
try:
    # Intentar con estratificación
    X_temp, X_test, y_temp, y_test, bbox_temp, bbox_test = train_test_split(
        images, labels, bboxes, test_size=0.15, random_state=42, stratify=labels
    )
    
    X_train, X_val, y_train, y_val, bbox_train, bbox_val = train_test_split(
        X_temp, y_temp, bbox_temp, test_size=0.18, random_state=42, stratify=y_temp
    )
    print("\n✅ División estratificada exitosa!")
    
except ValueError as e:
    print(f"\n⚠️  Estratificación falló: {e}")
    print("Usando división aleatoria sin estratificación...")
    
    # División sin estratificación
    X_temp, X_test, y_temp, y_test, bbox_temp, bbox_test = train_test_split(
        images, labels, bboxes, test_size=0.15, random_state=42
    )
    
    X_train, X_val, y_train, y_val, bbox_train, bbox_val = train_test_split(
        X_temp, y_temp, bbox_temp, test_size=0.18, random_state=42
    )
    print("✅ División aleatoria exitosa!")

print(f"\n📊 DIVISIÓN DEL DATASET:")
print(f"   - Entrenamiento: {len(X_train)} muestras ({len(X_train)/len(images)*100:.1f}%)")
print(f"   - Validación: {len(X_val)} muestras ({len(X_val)/len(images)*100:.1f}%)")
print(f"   - Prueba: {len(X_test)} muestras ({len(X_test)/len(images)*100:.1f}%)")


🔍 Analizando distribución de clases...
Classes with minimum samples (10):
   - Clases originales: 316
   - Clases filtradas (>=10 muestras): 147
   - Top clases: ['Person', 'Table', 'Building', 'Car', 'Taxi', 'Tie', 'Suit', 'Shotgun', 'Helmet', 'Curtain', 'Human arm', 'Backpack', 'Vehicle', 'Tree', 'Footwear', 'Shirt', 'Human ear', 'Human head', 'Toy', 'Book']

🎯 Configuración actualizada:
   - Resolución: 224x224
   - Número de clases: 25
   - Clases seleccionadas: ['Person', 'Table', 'Building', 'Car', 'Taxi', 'Tie', 'Suit', 'Shotgun', 'Helmet', 'Curtain', 'Human arm', 'Backpack', 'Vehicle', 'Tree', 'Footwear', 'Shirt', 'Human ear', 'Human head', 'Toy', 'Book', 'Human body', 'Handbag', 'Human face', 'Human hand', 'Window']

🔄 Procesando dataset con clases filtradas...
   Procesadas 100 imágenes...
   Procesadas 200 imágenes...
   Procesadas 300 imágenes...
   Procesadas 400 imágenes...
   Procesadas 500 imágenes...
   Procesadas 600 imágenes...
   Procesadas 700 imágenes...
   Proces

In [21]:
# 🚀 FASE 2: ARQUITECTURA MEJORADA CON TRANSFER LEARNING
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

print("🏗️  CONSTRUYENDO MODELO MEJORADO PARA TIEMPO REAL")
print("="*60)

def build_robust_detector(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=len(NEW_CLASSES)):
    """
    Modelo robusto para detección en tiempo real usando ResNet50V2
    - Transfer Learning con ResNet50V2 (más estable)
    - Arquitectura optimizada para velocidad y precisión
    - Dual-head: clasificación + bounding box regression
    """
    
    inputs = Input(shape=input_shape, name="input_image")
    
    # 🎯 BACKBONE: ResNet50V2 (más estable que EfficientNet para este caso)
    backbone = ResNet50V2(
        weights='imagenet',  # Pesos pre-entrenados
        include_top=False,   # Sin capa de clasificación final
        input_tensor=inputs,
        pooling='avg'        # Global Average Pooling
    )
    
    # Congelar las primeras capas del backbone (fine-tuning)
    for layer in backbone.layers[:-15]:  # Descongelar las últimas 15 capas
        layer.trainable = False
    
    x = backbone.output
    
    # 🔧 CABEZAS ESPECIALIZADAS
    # Feature enhancement con regularización
    x = layers.Dense(512, activation='relu', name='feature_dense')(x)
    x = layers.Dropout(0.4, name='feature_dropout')(x)
    x = layers.BatchNormalization(name='feature_bn')(x)
    
    # Segunda capa de características
    x = layers.Dense(256, activation='relu', name='feature_dense_2')(x)
    x = layers.Dropout(0.3, name='feature_dropout_2')(x)
    
    # 🎯 CABEZA DE CLASIFICACIÓN
    class_features = layers.Dense(128, activation='relu', name='class_features')(x)
    class_features = layers.Dropout(0.2, name='class_dropout')(class_features)
    class_output = layers.Dense(
        num_classes, 
        activation='softmax', 
        name='class_output'
    )(class_features)
    
    # 📦 CABEZA DE BOUNDING BOX
    bbox_features = layers.Dense(128, activation='relu', name='bbox_features')(x)
    bbox_features = layers.Dropout(0.2, name='bbox_dropout')(bbox_features)
    bbox_output = layers.Dense(
        4, 
        activation='sigmoid',  # Coordenadas normalizadas [0,1]
        name='bbox_output'
    )(bbox_features)
    
    model = models.Model(inputs=inputs, outputs=[class_output, bbox_output], name='RobustDetector')
    
    return model

# Construir modelo
print("🔧 Construyendo RobustDetector con ResNet50V2...")
try:
    model = build_robust_detector()
    print("✅ Modelo construido exitosamente!")
    
    # Resumen del modelo
    print(f"\n📊 ARQUITECTURA DEL MODELO:")
    print(f"   - Backbone: ResNet50V2 (pre-entrenado)")
    print(f"   - Parámetros totales: {model.count_params():,}")
    print(f"   - Parámetros entrenables: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
    print(f"   - Clases de salida: {len(NEW_CLASSES)}")
    
    # Mostrar resumen completo
    model.summary()
    
except Exception as e:
    print(f"❌ Error construyendo modelo con ResNet50V2: {e}")
    print("\n🔄 Intentando con arquitectura CNN personalizada...")
    
    # PLAN B: CNN personalizada pero robusta
    def build_custom_detector(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=len(NEW_CLASSES)):
        """CNN personalizada optimizada para tiempo real"""
        
        inputs = Input(shape=input_shape, name="input_image")
        
        # Bloque 1
        x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Dropout(0.25)(x)
        
        # Bloque 2
        x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Dropout(0.25)(x)
        
        # Bloque 3
        x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
        x = layers.Dropout(0.25)(x)
        
        # Bloque 4
        x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.5)(x)
        
        # Características densas
        x = layers.Dense(512, activation='relu', name='feature_dense')(x)
        x = layers.Dropout(0.4, name='feature_dropout')(x)
        x = layers.BatchNormalization(name='feature_bn')(x)
        
        # 🎯 CABEZA DE CLASIFICACIÓN
        class_features = layers.Dense(256, activation='relu', name='class_features')(x)
        class_features = layers.Dropout(0.3, name='class_dropout')(class_features)
        class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(class_features)
        
        # 📦 CABEZA DE BOUNDING BOX
        bbox_features = layers.Dense(256, activation='relu', name='bbox_features')(x)
        bbox_features = layers.Dropout(0.3, name='bbox_dropout')(bbox_features)
        bbox_output = layers.Dense(4, activation='sigmoid', name='bbox_output')(bbox_features)
        
        model = models.Model(inputs=inputs, outputs=[class_output, bbox_output], name='CustomDetector')
        return model
    
    model = build_custom_detector()
    print("✅ Modelo CNN personalizado construido exitosamente!")
    
    print(f"\n📊 ARQUITECTURA DEL MODELO:")
    print(f"   - Backbone: CNN Personalizada")
    print(f"   - Parámetros totales: {model.count_params():,}")
    print(f"   - Parámetros entrenables: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
    print(f"   - Clases de salida: {len(NEW_CLASSES)}")


🏗️  CONSTRUYENDO MODELO MEJORADO PARA TIEMPO REAL
🔧 Construyendo RobustDetector con ResNet50V2...
✅ Modelo construido exitosamente!

📊 ARQUITECTURA DEL MODELO:
   - Backbone: ResNet50V2 (pre-entrenado)
   - Parámetros totales: 24,816,797
   - Parámetros entrenables: 5,719,709
   - Clases de salida: 25


Model: "RobustDetector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_image[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 24,816,797 (94.67 MB)

 Trainable params: 5,719,709 (21.82 MB)

 Non-trainable params: 19,097,088 (72.85 MB)

In [22]:
# 🎯 ENTRENAMIENTO OPTIMIZADO PARA TIEMPO REAL
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
import matplotlib.pyplot as plt

print("⚡ CONFIGURANDO ENTRENAMIENTO OPTIMIZADO")
print("="*50)

# 🔧 COMPILACIÓN CON CONFIGURACIÓN OPTIMIZADA
model.compile(
    optimizer=Adam(learning_rate=0.001, decay=1e-6),
    loss={
        'class_output': 'sparse_categorical_crossentropy',
        'bbox_output': Huber(delta=1.0)  # Más robusta que MSE para bounding boxes
    },
    loss_weights={
        'class_output': 1.0,
        'bbox_output': 3.0  # Mayor peso para precisión de localización
    },
    metrics={
        'class_output': ['accuracy'],
        'bbox_output': ['mae']
    }
)

# 📚 CALLBACKS PARA ENTRENAMIENTO EFICIENTE
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Modelo compilado y listo para entrenamiento!")
print(f"\n📋 CONFIGURACIÓN DE ENTRENAMIENTO:")
print(f"   - Optimizador: Adam (lr=0.001)")
print(f"   - Loss clasificación: Sparse Categorical Crossentropy")
print(f"   - Loss bounding box: Huber Loss (más robusto)")
print(f"   - Peso de localización: 3x más importante")
print(f"   - Early Stopping: 5 épocas de paciencia")
print(f"   - ReduceLR: Factor 0.5 cada 3 épocas sin mejora")

# 🚀 ENTRENAMIENTO
print(f"\n🚀 INICIANDO ENTRENAMIENTO...")
print(f"   - Datos de entrenamiento: {len(X_train):,} muestras")
print(f"   - Datos de validación: {len(X_val):,} muestras")
print(f"   - Épocas máximas: 20")

history = model.fit(
    X_train,
    {'class_output': y_train, 'bbox_output': bbox_train},
    validation_data=(X_val, {'class_output': y_val, 'bbox_output': bbox_val}),
    epochs=25,
    batch_size=32,  # Batch size optimizado para velocidad
    callbacks=callbacks,
    verbose=1
)

print("\n🎉 ¡ENTRENAMIENTO COMPLETADO!")


⚡ CONFIGURANDO ENTRENAMIENTO OPTIMIZADO
✅ Modelo compilado y listo para entrenamiento!

📋 CONFIGURACIÓN DE ENTRENAMIENTO:
   - Optimizador: Adam (lr=0.001)
   - Loss clasificación: Sparse Categorical Crossentropy
   - Loss bounding box: Huber Loss (más robusto)
   - Peso de localización: 3x más importante
   - Early Stopping: 5 épocas de paciencia
   - ReduceLR: Factor 0.5 cada 3 épocas sin mejora

🚀 INICIANDO ENTRENAMIENTO...
   - Datos de entrenamiento: 6,114 muestras
   - Datos de validación: 1,343 muestras
   - Épocas máximas: 20


d:\Todo\Documentos\Santiago\3_Trabajo Profesional\Full-Stack-Projects\object-recognition-model\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/25
192/192 ━━━━━━━━━━━━━━━━━━━━ 241s 1s/step - bbox_output_loss: 0.0408 - bbox_output_mae: 0.2354 - class_output_accuracy: 0.5070 - class_output_loss: 1.8074 - loss: 1.9215 - val_bbox_output_loss: 0.0365 - val_bbox_output_mae: 0.2260 - val_class_output_accuracy: 0.6292 - val_class_output_loss: 1.3430 - val_loss: 1.4529 - learning_rate: 0.0010
Epoch 2/25
192/192 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - bbox_output_loss: 0.0349 - bbox_output_mae: 0.2205 - class_output_accuracy: 0.6122 - class_output_loss: 1.3142 - loss: 1.4127 - val_bbox_output_loss: 0.0346 - val_bbox_output_mae: 0.2194 - val_class_output_accuracy: 0.6418 - val_class_output_loss: 1.2748 - val_loss: 1.3787 - learning_rate: 0.0010
Epoch 3/25
192/192 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - bbox_output_loss: 0.0341 - bbox_output_mae: 0.2176 - class_output_accuracy: 0.6549 - class_output_loss: 1.1378 - loss: 1.2390 - val_bbox_output_loss: 0.0335 - val_bbox_output_mae: 0.2163 - val_class_output_accuracy: 0.6649 - val_class_outp

In [23]:
# 📹 FASE 3: IMPLEMENTACIÓN DE CÁMARA EN TIEMPO REAL
import cv2
import time
import numpy as np

print("📹 SISTEMA DE RECONOCIMIENTO EN TIEMPO REAL")
print("="*55)

class RealTimeObjectDetector:
    def __init__(self, model, class_names, img_size=224):
        self.model = model
        self.class_names = class_names
        self.img_size = img_size
        self.fps_counter = 0
        self.fps_start_time = time.time()
        self.current_fps = 0
        
    def preprocess_frame(self, frame):
        """Preprocesar frame para el modelo"""
        # Redimensionar manteniendo aspect ratio
        h, w = frame.shape[:2]
        scale = min(self.img_size/w, self.img_size/h)
        new_w, new_h = int(w*scale), int(h*scale)
        
        # Redimensionar y pad
        resized = cv2.resize(frame, (new_w, new_h))
        padded = np.zeros((self.img_size, self.img_size, 3), dtype=np.uint8)
        
        # Centrar la imagen
        y_offset = (self.img_size - new_h) // 2
        x_offset = (self.img_size - new_w) // 2
        padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
        
        # Normalizar
        processed = padded.astype(np.float32) / 255.0
        return np.expand_dims(processed, axis=0), scale, x_offset, y_offset
    
    def predict_frame(self, frame):
        """Hacer predicción en un frame"""
        processed, scale, x_offset, y_offset = self.preprocess_frame(frame)
        
        # Predicción
        pred_class, pred_bbox = self.model.predict(processed, verbose=0)
        
        # Procesar resultados
        class_idx = np.argmax(pred_class[0])
        confidence = pred_class[0][class_idx]
        bbox = pred_bbox[0]
        
        # Convertir bounding box a coordenadas originales
        orig_h, orig_w = frame.shape[:2]
        
        # Coordenadas relativas al frame procesado
        ymin, xmin, ymax, xmax = bbox
        
        # Ajustar por padding y escala
        xmin = (xmin * self.img_size - x_offset) / scale
        ymin = (ymin * self.img_size - y_offset) / scale
        xmax = (xmax * self.img_size - x_offset) / scale
        ymax = (ymax * self.img_size - y_offset) / scale
        
        # Clamping
        xmin = max(0, min(xmin, orig_w-1))
        ymin = max(0, min(ymin, orig_h-1))
        xmax = max(0, min(xmax, orig_w-1))
        ymax = max(0, min(ymax, orig_h-1))
        
        return {
            'class': self.class_names[class_idx],
            'confidence': confidence,
            'bbox': [int(xmin), int(ymin), int(xmax), int(ymax)]
        }
    
    def draw_prediction(self, frame, prediction, min_confidence=0.3):
        """Dibujar predicción en el frame"""
        if prediction['confidence'] < min_confidence:
            return frame
            
        # Extraer datos
        class_name = prediction['class']
        confidence = prediction['confidence']
        x1, y1, x2, y2 = prediction['bbox']
        
        # Colores para diferentes clases
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), 
                  (255, 0, 255), (0, 255, 255), (128, 0, 128), (255, 165, 0)]
        color = colors[hash(class_name) % len(colors)]
        
        # Dibujar bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Texto
        label = f"{class_name}: {confidence:.2f}"
        
        # Fondo para el texto
        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        cv2.rectangle(frame, (x1, y1-30), (x1+label_size[0], y1), color, -1)
        
        # Texto
        cv2.putText(frame, label, (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return frame
    
    def calculate_fps(self):
        """Calcular FPS"""
        self.fps_counter += 1
        if self.fps_counter >= 10:
            current_time = time.time()
            self.current_fps = 10 / (current_time - self.fps_start_time)
            self.fps_start_time = current_time
            self.fps_counter = 0
        return self.current_fps
    
    def run_webcam_detection(self, camera_id=0, duration=30):
        """Ejecutar detección en tiempo real desde webcam"""
        print(f"🎥 Iniciando cámara {camera_id}...")
        
        cap = cv2.VideoCapture(camera_id)
        if not cap.isOpened():
            print("❌ Error: No se pudo abrir la cámara")
            return
        
        # Configurar cámara
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        print(f"✅ Cámara iniciada. Detectando por {duration} segundos...")
        print("   Presiona 'q' para salir antes")
        
        start_time = time.time()
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("❌ Error leyendo frame")
                    break
                
                # Hacer predicción
                prediction = self.predict_frame(frame)
                
                # Dibujar resultados
                frame = self.draw_prediction(frame, prediction)
                
                # Mostrar FPS
                fps = self.calculate_fps()
                cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                # Mostrar frame
                cv2.imshow('Real-Time Object Detection', frame)
                
                # Verificar salida
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
                # Límite de tiempo
                if time.time() - start_time > duration:
                    print(f"⏰ Tiempo completado: {duration} segundos")
                    break
                    
        except KeyboardInterrupt:
            print("\n⏹️  Detección interrumpida por usuario")
        
        finally:
            cap.release()
            cv2.destroyAllWindows()
            print("✅ Cámara cerrada")

# Crear detector
detector = RealTimeObjectDetector(model, NEW_CLASSES, IMG_SIZE)

print("🚀 ¡DETECTOR EN TIEMPO REAL LISTO!")
print(f"\n📋 CARACTERÍSTICAS:")
print(f"   - Modelo: EfficientNetB0 + Transfer Learning")
print(f"   - Clases detectables: {len(NEW_CLASSES)}")
print(f"   - Resolución de entrada: {IMG_SIZE}x{IMG_SIZE}")
print(f"   - Optimizado para velocidad y precisión")
print(f"\n🎯 PARA USAR LA CÁMARA, EJECUTA:")
print(f"   detector.run_webcam_detection(camera_id=0, duration=60)")


📹 SISTEMA DE RECONOCIMIENTO EN TIEMPO REAL
🚀 ¡DETECTOR EN TIEMPO REAL LISTO!

📋 CARACTERÍSTICAS:
   - Modelo: EfficientNetB0 + Transfer Learning
   - Clases detectables: 25
   - Resolución de entrada: 224x224
   - Optimizado para velocidad y precisión

🎯 PARA USAR LA CÁMARA, EJECUTA:
   detector.run_webcam_detection(camera_id=0, duration=60)


In [24]:
print("🎥 Iniciando sistema de cámara...")

try:
    # Crear detector
    detector = RealTimeObjectDetector(model, NEW_CLASSES, IMG_SIZE)
    
    print("✅ Detector creado exitosamente!")
    print(f"📹 Iniciando cámara para detección...")
    print("💡 Presiona 'q' para salir")
    
    # Iniciar detección (60 segundos)
    detector.run_webcam_detection(camera_id=0, duration=180)
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("🔧 Verifica que:")
    print("   1. El modelo esté entrenado")
    print("   2. La cámara esté disponible")
    print("   3. Tengas permisos de cámara")

🎥 Iniciando sistema de cámara...
✅ Detector creado exitosamente!
📹 Iniciando cámara para detección...
💡 Presiona 'q' para salir
🎥 Iniciando cámara 0...
✅ Cámara iniciada. Detectando por 180 segundos...
   Presiona 'q' para salir antes

⏹️  Detección interrumpida por usuario
✅ Cámara cerrada
